# Sentence embeddings

In [3]:
import sentence_transformers
import numpy as np
from tqdm import tqdm_notebook

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

## Clean data and comments
- For data:
    - read id, title, selftext, and score
    - remove [removed] and [deleted], drop NAs, and strip white spaces.
- For comments:
    - read data and remove strings in score column
    - read parent_id, body, and score
    - remove [removed] and [deleted], drop NAs, and strip white spaces.
    - filter out t1_ comments. These are indirect comments.
    - keep t3_ id, and remain body and score.

In [5]:
import pandas as pd

def clean_data(csv_in):
    csv_read = pd.read_csv(csv_in)

    csv = csv_read[['id', 'title', 'selftext', 'score']]
    csv = csv[csv.selftext != '[removed]']
    csv = csv[csv.selftext != '[deleted]']
    csv.dropna(subset = ["selftext"], inplace=True)
    csv = csv.replace(r'^\s+','', regex=True)
    return csv

nosleep2020 = clean_data('./Creepy Data/NoSleep/RS_2020_nosleep.csv')

confessions2020 = clean_data('./Creepy Data/Confessions/RS_2020_confessions.csv')

In [6]:
def clean_comments(csv_in):
    csv_read = pd.read_csv(csv_in, dtype = str)
    csv = csv_read[csv_read['score'].apply(pd.to_numeric, errors='coerce').notna()]
    csv = csv[['parent_id', 'body', 'score']]
    csv = csv[csv.body != '[removed]']
    csv = csv[csv.body != '[deleted]']
    csv.dropna(subset = ["body"], inplace=True)
    csv = csv.replace(r'^\s+','', regex=True)
    
    # t3_ are direct comments
    csv = csv[csv.parent_id.str.startswith('t3_')]
    parent_id = csv.parent_id.str.split(r"_", expand=True)
    
    result = pd.concat([csv, parent_id], axis=1)
    result = result[[1, 'body', 'score']]
    result.columns = ['parent_id', 'body', 'score']
    return result
#     return csv
    
nosleep_comments = clean_comments('./Creepy Data/NoSleep/nosleep_comments.csv')
confessions_comments = clean_comments('./Creepy Data/Confessions/confessions_comments.csv')

## Pickle dfs for later use

In [46]:
nosleep2020.to_pickle('./pickles/nosleep2020.pickle')
confessions2020.to_pickle('./pickles/confessions2020.pickle')
nosleep_comments.to_pickle('./pickles/nosleep_comments.pickle')
confessions_comments.to_pickle('./pickles/confessions_comments.pickle')

## Using spaCy to separate sentences and sentence_transformers to get sentence vectors.

## Find linear combinations of r/NoSleep 's comments, grouped by parent_id

In [22]:
import spacy

spacy.prefer_gpu()
# Must return True for GPU to work

True

In [23]:
nlp = spacy.load('en_core_web_lg')

In [24]:
# need nlp and model
def sum_vec(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    comment_vec = model.encode(sentences)
    return np.sum(comment_vec, axis = 0)

In [25]:
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
tqdm.pandas()

/home/anthony/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [26]:
nosleep_comments.loc[:,'sum_vec'] = nosleep_comments.progress_apply(lambda x: sum_vec(x['body']), axis=1)

nosleep_comments.loc[:,'score'] = pd.to_numeric(nosleep_comments.loc[:,'score'])

# print(nosleep_comments_short.score.dtypes)
nosleep_comments['lc'] = nosleep_comments['score'].to_numpy() * nosleep_comments['sum_vec'].to_numpy()
nosleep_comments.groupby(['parent_id'])['lc'].apply(np.sum)

100%|██████████| 100000/100000 [29:21<00:00, 56.78it/s]


parent_id
1i0ws2    [-1.0927082, 0.8121819, 0.30228412, -0.0302171...
btkvkl    [0.2768101, 0.30319256, 0.36772513, -0.1068986...
bue2kk    [-0.35299757, -0.4115251, 0.013481392, 0.11935...
bvy4vv    [-0.32673618, -0.5601439, 0.122779846, 0.27947...
bw5p68    [1.0078417, 1.7216101, -0.14380181, -0.1225537...
                                ...                        
gxrnj7    [-0.42676052, -0.80316484, 0.20431642, -0.4659...
gxrudc    [-1.0927082, 0.8121819, 0.30228412, -0.0302171...
gxs69h    [-1.0927082, 0.8121819, 0.30228412, -0.0302171...
gxs6jf    [-1.245924, 5.5134716, 2.124174, -4.7967, -1.8...
gxsa0i    [-4.0768266, 1.1921716, 1.8977762, -2.6812117,...
Name: lc, Length: 23223, dtype: object

In [47]:
nosleep_comments

,parent_id,body,score,sum_vec,lc
4,gxq0yh,This is truly fascinating!! \n\nConsider makin...,1,"[0.58918774, -0.41969723, 0.74375904, -0.36868...","[0.58918774, -0.41969723, 0.74375904, -0.36868..."
5,gxg5z1,just.... no,2,"[-0.87192535, -0.15856919, -0.4702785, 0.14323...","[-1.7438507, -0.31713837, -0.940557, 0.2864617..."
6,gxjuvy,Im tearing up oh God youre such a lovely soul,2,"[-0.7478468, 0.63004714, 0.34330124, -0.496975...","[-1.4956936, 1.2600943, 0.6866025, -0.99395, -..."
8,gp0040,Okay - had to pause in my reading for a moment...,1,"[-0.06602657, 0.3970722, 0.1914272, -0.9652371...","[-0.06602657, 0.3970722, 0.1914272, -0.9652371..."
9,gxieer,I thought it was a cheerful story till the las...,2,"[-0.051630713, -0.75132173, 1.4272715, -0.7462...","[-0.103261426, -1.5026435, 2.854543, -1.492407..."
...,...,...,...,...,...
193439,dxez8f,Nice story but It would be nice if Xavier had ...,1,"[-1.0226526, -1.2354299, 0.20767829, -1.026228...","[-1.0226526, -1.2354299, 0.20767829, -1.026228..."
193440,dxi4ex,Have you tried talking to her/it at all?,1,"[-0.7380303, 0.26792833, -0.20137416, 0.054930...","[-0.7380303, 0.26792833, -0.20137416, 0.054930..."
193441,dxlgac,Your submission has been removed. Stories in /...,1,"[-1.0927082, 0.8121819, 0.30228412, -0.0302171...","[-1.0927082, 0.8121819, 0.30228412, -0.0302171..."
193443,dx7fft,Good night honey! We love you!,1,"[0.0048104525, 1.014739, 0.36305526, -0.773156...","[0.0048104525, 1.014739, 0.36305526, -0.773156..."


In [ ]:
nosleep_comments.to_pickle('./pickles/nosleep_comments.pickle')

### Find sum_vec of r/NoSleep

In [48]:
nosleep2020

,id,title,selftext,score
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1
6,gxrm8v,I'm tasked with killing nameless things out in...,“Any sign of ‘em yet?” \n\nI continued staring...,1
...,...,...,...,...
21218,eihp0m,Hylophobia,*There is no cure for trauma. Once it enters t...,1
21219,eihmg7,I adopted my late sisters orphaned child. This...,"I knew Persephone would need time to adjust, b...",1
21221,eihgtp,My first paranormal experience!!,"This isnt much, but this is surely the first u...",1
21223,eigzgj,I met the demon under my bed... Its not what I...,"Okay. for context, this story started about a ...",1


In [29]:
nosleep2020.loc[:,'sum_vec'] = nosleep2020.progress_apply(lambda x: sum_vec(x['selftext']), axis=1)

100%|██████████| 1000/1000 [02:44<00:00,  6.07it/s]


In [56]:
nosleep2020_short

,id,title,selftext,score,sum_vec
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1,"[-19.201273, -13.715499, 24.393753, -33.97739,..."
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,"[-60.77887, -33.726135, 119.47121, -95.021385,..."
6,gxrm8v,I'm tasked with killing nameless things out in...,“Any sign of ‘em yet?” \n\nI continued staring...,1,"[-22.785084, 14.806147, 26.129469, -24.832222,..."
...,...,...,...,...,...
1418,grk9vy,I am staying with an exchange family in Japan....,[Part 4](https://www.reddit.com/r/nosleep/comm...,1,"[-47.53694, 22.346678, 39.205128, -57.237534, ..."
1420,grjxp5,My little hobby helps me being a better mother,I’m first and foremost a mother.\n\nMy whole l...,1,"[4.022182, 6.498223, 19.040422, -20.935751, 3...."
1422,grjm41,Where Is Damien?,The young woman woke up in her apartment relax...,1,"[-35.111553, -9.009447, 25.581781, -40.168076,..."
1424,grja9o,I pay my parents an allowance every week. They...,I almost relish the pleading look in my dad's ...,1,"[-11.511959, -8.483317, 11.048408, -26.508553,..."


In [ ]:
nosleep2020.to_pickle('./pickles/nosleep2020.pickle')

In [31]:
nosleep_merged = nosleep_comments.merge(nosleep2020, left_on='parent_id', right_on='id')

In [50]:
nosleep_merged

,parent_id,comment,c_score,c_sum_vec,c_lc,id,title,selftext,t_score,t_sum_vec
0,gxq0yh,This is truly fascinating!! \n\nConsider makin...,1,"[0.58918774, -0.41969723, 0.74375904, -0.36868...","[0.58918774, -0.41969723, 0.74375904, -0.36868...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
1,gxq0yh,Woah...just woah. I got questions but too spee...,1,"[0.20346296, -0.94261086, 0.28327927, 0.676400...","[0.20346296, -0.94261086, 0.28327927, 0.676400...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
2,gxq0yh,If you or someone you know is contemplating su...,1,"[0.68557304, -0.67809576, 1.6547089, -3.315929...","[0.68557304, -0.67809576, 1.6547089, -3.315929...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
3,gxg5z1,just.... no,2,"[-0.87192535, -0.15856919, -0.4702785, 0.14323...","[-1.7438507, -0.31713837, -0.940557, 0.2864617...",gxg5z1,A man knocked on my door and offered me 10% of...,"“Hi, uh, I’m Rob and I was wondering if you ar...",1,"[-10.211775, 5.102462, 7.233557, -28.880627, 0..."
4,gxg5z1,I hope you will update us on your situation!!!...,1,"[-1.2264739, 0.58753633, 0.33913088, -0.622941...","[-1.2264739, 0.58753633, 0.33913088, -0.622941...",gxg5z1,A man knocked on my door and offered me 10% of...,"“Hi, uh, I’m Rob and I was wondering if you ar...",1,"[-10.211775, 5.102462, 7.233557, -28.880627, 0..."
...,...,...,...,...,...,...,...,...,...,...
4275,grlqs6,Hi there! [Your post](https://redd.it/grlqs6) ...,1,"[-1.9341251, 5.5325713, 3.9762313, -4.6057262,...","[-1.9341251, 5.5325713, 3.9762313, -4.6057262,...",grlqs6,Something is wrong with Splatoon2.,Okay. This happened last year and I just got R...,1,"[-15.509618, -29.070608, 23.578672, -17.360935..."
4276,grlpgz,Hi there! [Your post](https://redd.it/grlpgz) ...,1,"[-2.0490873, 5.406688, 4.1395907, -4.687119, -...","[-2.0490873, 5.406688, 4.1395907, -4.687119, -...",grlpgz,Missing Persons: The Cottage,Missing Persons Case File #9363446\n\nAudio Tr...,1,"[-97.37155, 59.17433, 84.36566, -58.829197, -2..."
4277,grl3np,If you or someone you know is contemplating su...,1,"[0.68557304, -0.67809576, 1.6547089, -3.315929...","[0.68557304, -0.67809576, 1.6547089, -3.315929...",grl3np,Puppet Spiders: Girlfriend,The following was recovered from an open docum...,1,"[-7.057307, -27.757399, 61.211403, -110.58104,..."
4278,grkthn,Hi there! [Your post](https://redd.it/grkthn) ...,1,"[-0.87736535, 2.380375, 3.5181599, -3.5410833,...","[-0.87736535, 2.380375, 3.5181599, -3.5410833,...",grkthn,It was Just the Other Day,"&amp;#x200B;\n\n It was September 23, 2019. I ...",1,"[89.26948, -82.584015, 119.83574, -157.07622, ..."


In [33]:
nosleep_merged.columns = ['parent_id', 'comment', 'c_score', 'c_sum_vec', 'c_lc', 'id', 'title', 'selftext', 't_score', 't_sum_vec']

In [34]:
len(nosleep_merged.id.unique())

824

# Groupby each post then by comments

In [51]:
# pd.set_option('display.max_rows', 50)
# nosleep_merged.loc[nosleep_merged.id == 'gxieer', :].groupby(['title', 'comment']).first()

In [52]:

comment_lc = pd.DataFrame(nosleep_merged.groupby(['id'])['c_lc'].apply(np.sum))
# comment_lc.reset_index(inplace=True) # Resets the index, makes factor a column
# comment_lc.drop("comment",axis=1,inplace=True) # drop factor from axis 1 and make changes permanent by inplace=True
comment_lc

,c_lc
id,
grih4t,"[19.786743, 0.19577622, 48.694035, -73.32118, ..."
grja9o,"[5.383287, 2.810141, -1.6748575, -9.672037, 5...."
grjxp5,"[0.54697853, -0.024439897, 0.59425366, -1.7169..."
grk9vy,"[-8.308628, 21.838442, 85.344315, -114.07018, ..."
grklwb,"[5.5355363, 1.8354892, 13.161881, -15.11415, -..."
...,...
gxr4ff,"[-3.4779508, 1.8638754, 3.4196403, -3.8453462,..."
gxr7hh,"[-3.3774014, -0.084626645, 3.8341122, -4.97267..."
gxrnj7,"[-0.42676052, -0.80316484, 0.20431642, -0.4659..."


In [53]:
nosleep_merged_gb_id = nosleep_merged.groupby(['id']).first()

In [54]:
# print(nosleep_comments_short.score.dtypes)
nosleep_merged_gb_id['t_lc'] = nosleep_merged_gb_id['t_score'].to_numpy() * nosleep_merged_gb_id['t_sum_vec'].to_numpy()
nosleep_merged_gb_id.groupby(['parent_id'])['t_lc'].apply(np.sum)

parent_id
grih4t    [-7.112637, -3.803339, 20.036457, -25.787561, ...
grja9o    [-11.511959, -8.483317, 11.048408, -26.508553,...
grjxp5    [4.022182, 6.498223, 19.040422, -20.935751, 3....
grk9vy    [-47.53694, 22.346678, 39.205128, -57.237534, ...
grklwb    [63.909054, 22.460388, 103.46454, -106.80335, ...
                                ...                        
gxr4ff    [-5.7507873, -26.91448, 14.6927805, -42.703568...
gxr7hh    [-3.1201575, 0.2122544, 12.899367, -13.549828,...
gxrnj7    [-60.77887, -33.726135, 119.47121, -95.021385,...
gxs6jf    [2.940986, -5.617336, 11.896912, -15.436118, -...
gxsa0i    [2.940986, -5.617336, 11.896912, -15.436118, -...
Name: t_lc, Length: 824, dtype: object

In [55]:
nosleep_merged_gb_id

,parent_id,comment,c_score,c_sum_vec,c_lc,title,selftext,t_score,t_sum_vec,t_lc
id,,,,,,,,,,
grih4t,grih4t,These stories are awesome! I must have more!!!,1,"[0.6580754, 0.511249, 0.59595877, -0.7810467, ...","[0.6580754, 0.511249, 0.59595877, -0.7810467, ...",I own a boutique that offers full body transfo...,I’m humbled that you all are so interested in ...,1,"[-7.112637, -3.803339, 20.036457, -25.787561, ...","[-7.112637, -3.803339, 20.036457, -25.787561, ..."
grja9o,grja9o,I saved 20% or more by switching to Geico!\n\n...,1,"[0.29487035, -1.2075154, -0.43160868, -0.98518...","[0.29487035, -1.2075154, -0.43160868, -0.98518...",I pay my parents an allowance every week. They...,I almost relish the pleading look in my dad's ...,1,"[-11.511959, -8.483317, 11.048408, -26.508553,...","[-11.511959, -8.483317, 11.048408, -26.508553,..."
grjxp5,grjxp5,o_O could see my mum doing this lmao,1,"[0.54697853, -0.024439897, 0.59425366, -1.7169...","[0.54697853, -0.024439897, 0.59425366, -1.7169...",My little hobby helps me being a better mother,I’m first and foremost a mother.\n\nMy whole l...,1,"[4.022182, 6.498223, 19.040422, -20.935751, 3....","[4.022182, 6.498223, 19.040422, -20.935751, 3...."
grk9vy,grk9vy,"The ""Grandmother"" reminds me of the monster fr...",1,"[0.5454855, -0.36329693, 0.780074, -1.5520129,...","[0.5454855, -0.36329693, 0.780074, -1.5520129,...",I am staying with an exchange family in Japan....,[Part 4](https://www.reddit.com/r/nosleep/comm...,1,"[-47.53694, 22.346678, 39.205128, -57.237534, ...","[-47.53694, 22.346678, 39.205128, -57.237534, ..."
grklwb,grklwb,Woo waa wee wooooow this took me in so many un...,1,"[-1.196348, -0.82951593, 4.3174777, -2.2829642...","[-1.196348, -0.82951593, 4.3174777, -2.2829642...",I took a job cleaning sewers for a private con...,"I’ve had a lot of dead-end jobs before, but ne...",1,"[63.909054, 22.460388, 103.46454, -106.80335, ...","[63.909054, 22.460388, 103.46454, -106.80335, ..."
...,...,...,...,...,...,...,...,...,...,...
gxr4ff,gxr4ff,"Well written, and an interesting story. Lookin...",1,"[0.1701939, -0.21244906, 1.1521204, -1.5114393...","[0.1701939, -0.21244906, 1.1521204, -1.5114393...",No one in my family is ever buried or cremated...,"It's a strict rule in my family, usually adher...",1,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-5.7507873, -26.91448, 14.6927805, -42.703568..."
gxr7hh,gxr7hh,"I know what you feel, not because I am in your...",1,"[-3.3774014, -0.084626645, 3.8341122, -4.97267...","[-3.3774014, -0.084626645, 3.8341122, -4.97267...",General Rantish,So basically i have just been feeling very dow...,1,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.1201575, 0.2122544, 12.899367, -13.549828,..."
gxrnj7,gxrnj7,...and just the revolution of an engine..,1,"[-0.42676052, -0.80316484, 0.20431642, -0.4659...","[-0.42676052, -0.80316484, 0.20431642, -0.4659...","I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-60.77887, -33.726135, 119.47121, -95.021385,..."


In [41]:
text_lc = nosleep_merged_gb_id.loc[:,['t_lc']]
text_lc

,t_lc
id,
grih4t,"[-7.112637, -3.803339, 20.036457, -25.787561, ..."
grja9o,"[-11.511959, -8.483317, 11.048408, -26.508553,..."
grjxp5,"[4.022182, 6.498223, 19.040422, -20.935751, 3...."
grk9vy,"[-47.53694, 22.346678, 39.205128, -57.237534, ..."
grklwb,"[63.909054, 22.460388, 103.46454, -106.80335, ..."
...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568..."
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,..."
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,..."


In [42]:
nosleep = pd.merge(text_lc, comment_lc, on='id', how = 'outer')

In [43]:
nosleep

,t_lc,c_lc
id,,
grih4t,"[-7.112637, -3.803339, 20.036457, -25.787561, ...","[19.786743, 0.19577622, 48.694035, -73.32118, ..."
grja9o,"[-11.511959, -8.483317, 11.048408, -26.508553,...","[5.383287, 2.810141, -1.6748575, -9.672037, 5...."
grjxp5,"[4.022182, 6.498223, 19.040422, -20.935751, 3....","[0.54697853, -0.024439897, 0.59425366, -1.7169..."
grk9vy,"[-47.53694, 22.346678, 39.205128, -57.237534, ...","[-8.308628, 21.838442, 85.344315, -114.07018, ..."
grklwb,"[63.909054, 22.460388, 103.46454, -106.80335, ...","[5.5355363, 1.8354892, 13.161881, -15.11415, -..."
...,...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-3.4779508, 1.8638754, 3.4196403, -3.8453462,..."
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.3774014, -0.084626645, 3.8341122, -4.97267..."
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-0.42676052, -0.80316484, 0.20431642, -0.4659..."


In [44]:
nosleep['output'] = nosleep.apply(np.sum, axis = 1)
nosleep['creepy'] = 1

In [45]:
nosleep

,t_lc,c_lc,output,creepy
id,,,,
grih4t,"[-7.112637, -3.803339, 20.036457, -25.787561, ...","[19.786743, 0.19577622, 48.694035, -73.32118, ...","[12.674107, -3.6075628, 68.73049, -99.10874, 1...",1
grja9o,"[-11.511959, -8.483317, 11.048408, -26.508553,...","[5.383287, 2.810141, -1.6748575, -9.672037, 5....","[-6.128672, -5.6731763, 9.37355, -36.180588, 1...",1
grjxp5,"[4.022182, 6.498223, 19.040422, -20.935751, 3....","[0.54697853, -0.024439897, 0.59425366, -1.7169...","[4.5691605, 6.473783, 19.634676, -22.652678, 3...",1
grk9vy,"[-47.53694, 22.346678, 39.205128, -57.237534, ...","[-8.308628, 21.838442, 85.344315, -114.07018, ...","[-55.84557, 44.18512, 124.54944, -171.30771, 1...",1
grklwb,"[63.909054, 22.460388, 103.46454, -106.80335, ...","[5.5355363, 1.8354892, 13.161881, -15.11415, -...","[69.44459, 24.295877, 116.62642, -121.9175, -1...",1
...,...,...,...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-3.4779508, 1.8638754, 3.4196403, -3.8453462,...","[-9.228738, -25.050606, 18.112421, -46.548912,...",1
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.3774014, -0.084626645, 3.8341122, -4.97267...","[-6.4975586, 0.12762776, 16.733479, -18.522501...",1
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-0.42676052, -0.80316484, 0.20431642, -0.4659...","[-61.20563, -34.5293, 119.67552, -95.48733, -1...",1
